In [1]:
import get_schedule
import pandas as pd
import numpy as np

In [2]:
freetime = get_schedule.Schedule().run(
        login_code="chla",
        start_date="2023-08-02",
        end_date="2023-08-07",
        names=['Benedetto, Lauren', "Freeburg, Taylor"]
    )

reading from cache


In [ ]:
availabilities = {
        "July 10": [
            {"start_time": 0, "end_time": 6, "display_range": "12:00AM - 6:00AM (6 hrs)"},
            {"start_time": 17, "end_time": 24, "display_range": "5:00PM - 11:59PM (7 hrs)"},
        ],
        "July 11": [
            {"start_time": 18, "end_time": 24, "display_range": "6:00PM - 11:59PM (6 hrs)"},
        ],
        "July 14": [
            {"start_time": 21, "end_time": 24, "display_range": "9:00PM - 11:59PM (3 hrs)"},
        ],
    }

In [13]:
availabilities = {}
prev_date = ""
for row in freetime.sort_values(by='date').iterrows():
    r = row[1]
    hours = int(r['end_time'] - r['start_time'])
    free_time_detail = {
            "start_time": r['start_time'], 
            "end_time": r['end_time'], 
            "display_range": f"{r['time_period']} ({str(hours)} hours)"
        }
    if r['date'] != prev_date:
        availabilities[r['date']] = [free_time_detail]
        prev_date = r['date']
    else:
        availabilities[r['date']].append(free_time_detail)
        prev_date = r['date']


In [14]:
availabilities

{'2023-08-02': [{'start_time': 0,
   'end_time': 17,
   'display_range': '12:00 AM to 5:00 PM (17 hours)'}],
 '2023-08-03': [{'start_time': 8,
   'end_time': 17,
   'display_range': '8:00 AM to 5:00 PM (9 hours)'}],
 '2023-08-04': [{'start_time': 8,
   'end_time': 17,
   'display_range': '8:00 AM to 5:00 PM (9 hours)'}],
 '2023-08-05': [{'start_time': 8,
   'end_time': 17,
   'display_range': '8:00 AM to 5:00 PM (9 hours)'}],
 '2023-08-06': [{'start_time': 13,
   'end_time': 23,
   'display_range': '1:00 PM to 11:59 PM (10 hours)'}]}

In [15]:
list(np.arange(24))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [22]:
eetime['hour'] == 23, "timestamp"] = freetime['timestamp'] + pd.Timedelta(minutes=59)
freetime['timestamp'] = freetime['timestamp'].dt.strftime("%-I:%M %p")
freetime = freetime.reset_index()
freetime_fwd = freetime.copy()
freetime_fwd['index'] = freetime_fwd['index'] + 1

ft = freetime.merge(
    freetime_fwd[['index', 'date', 'free_time']], 
    on = 'index', 
    how = 'left', 
    suffixes = ["", "_1hr_fwd"]
)

ft = ft.loc[
    (ft['date_1hr_fwd'].isnull()) 
    | ((ft['date'] == ft['date_1hr_fwd']) & (ft['free_time'] != ft['free_time_1hr_fwd']))
    | ((ft['hour'] == 23) & (ft['free_time'] == ft['free_time_1hr_fwd']))
    | (ft['hour'] == 0)
][['timestamp', 'date', 'hour', 'free_time']]

ct = 0
list_of_display_rows = []
for row in ft.to_dict("records"):
    if ct == 0 or row['hour'] == 0:
        time_start = str(row['timestamp'])
        free_at_start = row["free_time"]
        time_end = ""
        ct += 1
    elif time_end == "":
        time_end = str(row['timestamp'])
        free_at_end = row["free_time"]

        if free_at_start:
            status = "FREE"
        else:
            status = "BUSY"

        list_of_display_rows.append(
            pd.DataFrame(
                {
                    "status": [status],
                    "date": [str(row['date'])],
                    "time_period": [f"{time_start} to {time_end}"],
                }
            )
        )

        free_at_start = free_at_end
        time_start = time_end
        time_end = ""
        ct += 1
        
pd.concat(list_of_display_rows)

,status,date,time_period
0,FREE,2023-08-02,12:00 AM to 5:00 PM
0,BUSY,2023-08-02,5:00 PM to 11:59 PM
0,BUSY,2023-08-03,12:00 AM to 8:00 AM
0,FREE,2023-08-03,8:00 AM to 5:00 PM
0,BUSY,2023-08-03,5:00 PM to 11:59 PM
0,BUSY,2023-08-04,12:00 AM to 8:00 AM
0,FREE,2023-08-04,8:00 AM to 5:00 PM
0,BUSY,2023-08-04,5:00 PM to 11:59 PM
0,BUSY,2023-08-05,12:00 AM to 8:00 AM
0,FREE,2023-08-05,8:00 AM to 5:00 PM


In [26]:
df = pd.DataFrame({
    "name": ["tyler", "lauren", "taylor", "matt"]
})

In [33]:
"\r\n".join(list(df.name.unique())) + "\r\n"

'tyler\r\nlauren\r\ntaylor\r\nmatt\r\n'

In [36]:
names = list(df.name.unique())

In [39]:
string_of_names = ",".join(names)

In [42]:
list_of_files= string_of_names.split(",")
files = [(x, x) for x in list_of_files]

In [43]:
files

[('tyler', 'tyler'),
 ('lauren', 'lauren'),
 ('taylor', 'taylor'),
 ('matt', 'matt')]

In [45]:
list(np.arange(24))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [ ]:
ft = freetime.merge(
    freetime_fwd[['index', 'date', 'free_time']], 
    on = 'index', 
    how = 'left', 
    suffixes = ["", "_1hr_fwd"]
)

ft.loc[
    (ft['date_1hr_fwd'].isnull()) 
    | ((ft['date'] == ft['date_1hr_fwd']) & (ft['free_time'] != ft['free_time_1hr_fwd']))
    | ((ft['hour'] == 23) & (ft['free_time'] == ft['free_time_1hr_fwd']))
    | (ft['hour'] == 0),
    "flag"
] = "FLAG"

In [ ]:
ft[(ft['date'].astype(str).isin(["2023-08-02", "2023-08-03"])) & (ft['flag'].notnull()) ]

In [47]:
df = pd.read_csv(r"/Users/tylerfong/repos/residents-doing-things-flask/_cache/amion_cache_start=202382_days=14.csv")

In [49]:
df[df['name'].str.contains("Bruce")]

,name,name_id1,name_id2,team,team_id1,team_id2,date,start_time,end_time


In [16]:
import datetime as dt

In [22]:
dt.datetime.strptime("2023-08-01", "%Y-%m-%d").strftime("%B %-d")

'August 1'